In [0]:
%run "../../../config/setup"

In [0]:
CREATE OR REPLACE VIEW orders_dedupped AS
  SELECT 
    item_id, 
    transaction_id, 
    max(quantity) AS quantity, 
    max(price) AS price, 
    max(source_file) AS source_file, 
    max(ingest_ts) AS ingest_ts
  FROM orders_bronze
  WHERE 
    item_id IS NOT NULL AND
    transaction_id IS NOT NULL 
  GROUP BY item_id, transaction_id
  

In [0]:
GRANT SELECT ON VIEW orders_dedupped TO devs

In [0]:
CREATE OR REPLACE TABLE orders_silver AS
  SELECT 
    item_id, 
    transaction_id, 
    quantity, 
    price,
    CAST(ingest_ts AS DATE) AS ingest_date
  FROM orders_dedupped
  

In [0]:
GRANT SELECT ON TABLE orders_silver TO devs

In [0]:
-- fix error in generated orders and transaction data

-- SELECT 
--   a.transaction_id, 
--   a.item_id,
--   b.total_amount
-- FROM orders_silver AS a
-- INNER JOIN transactions_silver AS b
-- ON a.transaction_id = b.transaction_id
-- GROUP BY a.transaction_id, a.item_id, b.total_amount
-- HAVING SUM(a.quantity * a.price) = b.total_amount